In [1]:
# 1. Librerias
from pprint import pprint
import json
from openai import OpenAI
import time
from dotenv import load_dotenv
import os
from Funciones import gather_user_data, gather_reservation_data, not_talk
###################################################
# 2. Cargar variables entorno
load_dotenv();
openai_key= os.getenv("OPENAI_KEY");
#print(openai_key)
###################################################
# 3. Crear el cliente (permite la conexión con OPENAI)
client=OpenAI(api_key= openai_key);
###################################################
# 4. Obtener el asistente creado
assistant = client.beta.assistants.retrieve("asst_1IL5Wh27gRzwuIhCAmA0Gyww");